In [ ]:
import re
import usaddress
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import requests
from bs4 import BeautifulSoup

In [ ]:
# function to search keywords and scrape URL from google result page
def google_search(university_list,keyword_list):
    URL_list=pd.DataFrame(columns=('title',
    'link',
    'university',
    'keyword',
    'rank'))
    for university in university_list:
        for keyword in keyword_list:
            search_word='+'.join((university+'+'+keyword).split())
            URL = f"https://google.com/search?q={search_word}".format(search_word=search_word)
            resp = requests.get(URL, headers=headers)
            soup = BeautifulSoup(resp.content, "html.parser")
            rank_num = 1
            for g in soup.find_all('div', class_='r'):
                anchors = g.find_all('a')
                if anchors:
                    link = anchors[0]['href']
                    title = g.find('h3').text
                    item = {
                        "title": title,
                        "link": link,
                        'university':university,
                        'keyword':keyword,
                        'rank':rank_num
                        }
                    URL_list=URL_list.append(item,ignore_index=True)
                rank_num=rank_num+1
        URL_list.to_csv('URL_list.csv')
    return URL_list

In [ ]:
# function to parse address from webpage
def url_postal_adddress(url):
#     USER_AGENT = "Mozilla/5.0 (Linux; <Android Version>; <Build Tag etc.>) AppleWebKit/<WebKit Rev>(KHTML, like Gecko) Chrome/<Chrome Rev> Safari/<WebKit Rev>"
    USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
    headers = {"user-agent" : USER_AGENT}
    
    RE_address=re.compile(r"""(
                \d{1,5}[ ]    #street number
                (?:E\.[ ]|S\.[ ]|W\.[ ]|N\.[ ])?    #east south west north prefix
                [a-zA-Z ]{1,20}    #street name
                (?:street|st|avenue|ave|road|rd|highway|hwy|square|sq|trail|trl|drive|dr|court|ct|park|parkway|pkwy|circle|cir|boulevard|blvd)\W?(?=\s)\s*    #street surffix
                (?:\w{10})?
                [a-zA-Z\']{1,14}(?:\s[a-zA-Z\']{1,14}){0,1} #city
                (?:,\s|\b)?(?:AL|AK|AR|AZ|CA|CO|CT|DC|DE|FL|GA|HI|IA|ID|IL|IN|KS|KY|LA|MA|MD|ME|MI|MN|MO|MS|MT|NC|ND|NE|NH|NJ|NM|NV|NY|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VA|VT|WA|WI|WV|WY) #state
                (?:,\s|\s)?\b\d{5}(?:[-\s]\d{4})?\b
                 )""", re.IGNORECASE | re.VERBOSE)
    
    address_parsed_list=pd.DataFrame(columns=('AddressRaw',
    'AddressNumberPrefix',
    'AddressNumber',
    'AddressNumberSuffix',
    'StreetNamePreModifier',
    'StreetNamePreDirectional',
    'StreetNamePreType',
    'StreetName',
    'StreetNamePostType',
    'StreetNamePostDirectional',
    'SubaddressType',
    'SubaddressIdentifier',
    'BuildingName',
    'OccupancyType',
    'OccupancyIdentifier',
    'CornerOf',
    'LandmarkName',
    'PlaceName',
    'StateName',
    'ZipCode',
    'USPSBoxType',
    'USPSBoxID',
    'USPSBoxGroupType',
    'USPSBoxGroupID',
    'IntersectionSeparator',
    'Recipient'))
    
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")
    website_text=re.sub('\s+',' ',soup.text)
    address_list=re.findall(RE_address,website_text)
    
    for address in address_list:
        address_parsed=np.flip(usaddress.parse(address),1).T
        address_df = pd.DataFrame(data=address_parsed,columns=address_parsed[0,:])
        address_df = address_df[1:]
        address_df['AddressRaw']=address
        address_df = address_df.groupby(address_df.columns, axis=1).apply(lambda x: x.apply(lambda y: ' '.join([l for l in y if l is not None]), axis=1))
        address_parsed_list=address_parsed_list.append(address_df)
    return address_parsed_list

In [ ]:
#load university names and define keywords
university_list=pd.read_csv("universities.csv",encoding='latin1')['University_Name']
keyword_list=['dormitory address', 'residential hall']

In [ ]:
#search and scrape URL from google result page
url_list=google_search(university_list,keyword_list)
# url_list.to_csv('URL_list.csv')

In [ ]:
# for each URL, scrape and parse the address and store in URL_address_all
URL_address_all = pd.DataFrame()
URL_list_missed = pd.DataFrame()
for index, row in url_list.iterrows():
    try:
        row_df=pd.DataFrame(row).T
        row_df['join_key']=1
        address_df=url_postal_adddress(row['link'])
        address_df['join_key']=1
        URL_address=pd.merge(row_df,address_df).drop(columns=['Unnamed: 0','join_key'])
        if URL_address.size>0:
            if URL_address_all.size > 0:
                URL_address_all=URL_address_all.append(URL_address)
            else:
                URL_address_all=URL_address
        else:
            if URL_list_missed.size==0:
                URL_list_missed=row.to_frame().T
            else:
                URL_list_missed=URL_list_missed.append(row)
#         URL_address_all.to_csv('university_address.csv')
        print(URL_address.size)
    except:
        if URL_list_missed.size==0:
            URL_list_missed=row.to_frame()
        else:
            URL_list_missed=URL_list_missed.append(row)